In [1]:
import numpy as np
import pandas as pd
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Lets read the data and see what we're dealing with

In [4]:
file_location = 'data/Dam-levels-update-2012-2017_20171212.csv'
data = pd.read_csv(file_location, header = [2,3,4], encoding = 'latin-1')
data.head()

,Unnamed: 0_level_0,WEMMERSHOEK,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,STEENBRAS LOWER,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,STEENBRAS UPPER,...,Unnamed: 51_level_0,Unnamed: 52_level_0,TOTAL STORED - BIG 6,Unnamed: 54_level_0,Unnamed: 55_level_0,Unnamed: 56_level_0,LAND-en ZEEZICHT,Unnamed: 58_level_0,Unnamed: 59_level_0,Unnamed: 60_level_0
,DATE,HEIGHT,STORAGE,Current,Last Year,HEIGHT,STORAGE,Current,Last Year,HEIGHT,...,Current,Last Year,STORAGE,Current,Last Year,Unnamed: 56_level_1,HEIGHT,STORAGE,Current,Last Year
,Unnamed: 0_level_2,(m),(Ml),%,%,(m),(Ml),%,%,(m),...,%,%,(Ml),%,%,Unnamed: 56_level_2,(m),(Ml),%,%
0,01-Jan-12,48.23,44 621,76.1,NaN,20.34,23 549,70.3,NaN,24.17,...,89.2,NaN,695 783,77.46,NaN,NaN,NaN,NaN,NaN,NaN
1,02-Jan-12,48.21,44 571,76.0,NaN,20.31,23 460,70.0,NaN,24.17,...,88.3,NaN,692 990,77.15,NaN,NaN,NaN,NaN,NaN,NaN
2,03-Jan-12,48.17,44 471,75.8,NaN,20.28,23 372,69.7,NaN,24.15,...,87.9,NaN,690 324,76.85,NaN,NaN,NaN,NaN,NaN,NaN
3,04-Jan-12,48.13,44 372,75.7,NaN,20.26,23 313,69.6,NaN,24.17,...,87.5,NaN,687 282,76.52,NaN,NaN,NaN,NaN,NaN,NaN
4,05-Jan-12,48.11,44 322,75.6,NaN,20.23,23 224,69.3,NaN,24.05,...,87.2,NaN,683 355,76.08,NaN,NaN,NaN,NaN,NaN,NaN


# Collapsing the column names

First we're going to deal with the column names and 'flatten' them out into a single level. The columns names have been represented in the DataFrame in a MultiIndex, which allows for multiple levels of column names. Where it says 'Unnamed...', this represents a blank space in the original csv file.

We can also see that totals have been pre-calculated for the Big 6 dams. We don't want this is the final dataset.

In [6]:
data.columns

MultiIndex(levels=[['ALEXANDRA', 'BERG RIVER', 'DE VILLIERS', 'HELY-HUTCHINSON', 'KLEINPLAATS', 'LAND-en ZEEZICHT', 'LEWIS GAY', 'STEENBRAS LOWER', 'STEENBRAS UPPER', 'THEEWATERSKLOOF', 'TOTAL STORED - BIG 6', 'Unnamed: 0_level_0', 'Unnamed: 10_level_0', 'Unnamed: 11_level_0', 'Unnamed: 12_level_0', 'Unnamed: 14_level_0', 'Unnamed: 15_level_0', 'Unnamed: 16_level_0', 'Unnamed: 18_level_0', 'Unnamed: 19_level_0', 'Unnamed: 20_level_0', 'Unnamed: 22_level_0', 'Unnamed: 23_level_0', 'Unnamed: 24_level_0', 'Unnamed: 26_level_0', 'Unnamed: 27_level_0', 'Unnamed: 28_level_0', 'Unnamed: 2_level_0', 'Unnamed: 30_level_0', 'Unnamed: 31_level_0', 'Unnamed: 32_level_0', 'Unnamed: 34_level_0', 'Unnamed: 35_level_0', 'Unnamed: 36_level_0', 'Unnamed: 38_level_0', 'Unnamed: 39_level_0', 'Unnamed: 3_level_0', 'Unnamed: 40_level_0', 'Unnamed: 42_level_0', 'Unnamed: 43_level_0', 'Unnamed: 44_level_0', 'Unnamed: 46_level_0', 'Unnamed: 47_level_0', 'Unnamed: 48_level_0', 'Unnamed: 4_level_0', 'Unnamed: 50

In [8]:
# Break multi-level column names into individual lists
header_0 = [h[0] for h in data.columns[1:]]
header_1 = [h[1] for h in data.columns[1:]]
header_2 = [h[2] for h in data.columns[1:]]

# Fill 'Unnamed' top-level column names with dam names from left to right
for idx, h in enumerate(header_0):
    if re.search(r'^Unnamed', h) is not None:
        header_0[idx] = header_0[idx - 1]

header_0 = [h.title() for h in header_0]
header_1 = [h.lower() for h in header_1]
header_2 = [h.lower() for h in header_2]

col_names = ['date']
# Join all 3 levels of the column names together
col_names = col_names + list(map('_'.join, zip(header_0, header_1, header_2)))

data.columns = pd.Index(col_names)
# Get rid of 'totals'
data = data.drop([col for col in data.columns if re.match(r'^Total Stored', col)], axis = 1)

data['date'] = pd.to_datetime(data['date'])

In [9]:
data.head()

,date,Wemmershoek_height_(m),Wemmershoek_storage_(ml),Wemmershoek_current_%,Wemmershoek_last year_%,Steenbras Lower_height_(m),Steenbras Lower_storage_(ml),Steenbras Lower_current_%,Steenbras Lower_last year_%,Steenbras Upper_height_(m),...,Theewaterskloof_current_%,Theewaterskloof_last year_%,Berg River_height_(m),Berg River_storage_(ml),Berg River_current_%,Berg River_last year_%,Land-En Zeezicht_height_(m),Land-En Zeezicht_storage_(ml),Land-En Zeezicht_current_%,Land-En Zeezicht_last year_%
0,2012-01-01,48.23,44 621,76.1,NaN,20.34,23 549,70.3,NaN,24.17,...,74.5,NaN,39.33,115 930,89.2,NaN,NaN,NaN,NaN,NaN
1,2012-01-02,48.21,44 571,76.0,NaN,20.31,23 460,70.0,NaN,24.17,...,74.3,NaN,39.12,114 850,88.3,NaN,NaN,NaN,NaN,NaN
2,2012-01-03,48.17,44 471,75.8,NaN,20.28,23 372,69.7,NaN,24.15,...,74.0,NaN,39.01,114 280,87.9,NaN,NaN,NaN,NaN,NaN
3,2012-01-04,48.13,44 372,75.7,NaN,20.26,23 313,69.6,NaN,24.17,...,73.7,NaN,38.91,113 770,87.5,NaN,NaN,NaN,NaN,NaN
4,2012-01-05,48.11,44 322,75.6,NaN,20.23,23 224,69.3,NaN,24.05,...,73.1,NaN,38.84,113 410,87.2,NaN,NaN,NaN,NaN,NaN


# Dealing with dates

It's always a good idea to sense-check data. Date are especially susceptible to being messed up because of the various different ways that they can be formatted.

We can see below that the minimum date is 1 Jan 2012. This is fine. However, the maximum date is 22 May *2021*, which is obviously wrong, since that date hasn't yet occured. We are going to need to investigate what's going on here a little further.

In [212]:
data.shape
data['date'].min()
data['date'].max()

(2182, 57)

Timestamp('2012-01-01 00:00:00')

Timestamp('2021-05-22 00:00:00')

,date,Wemmershoek_height_(m),Wemmershoek_storage_(ml),Wemmershoek_current_%,Wemmershoek_last year_%,Steenbras Lower_height_(m),Steenbras Lower_storage_(ml),Steenbras Lower_current_%,Steenbras Lower_last year_%,Steenbras Upper_height_(m),...,Theewaterskloof_current_%,Theewaterskloof_last year_%,Berg River_height_(m),Berg River_storage_(ml),Berg River_current_%,Berg River_last year_%,Land-En Zeezicht_height_(m),Land-En Zeezicht_storage_(ml),Land-En Zeezicht_current_%,Land-En Zeezicht_last year_%
2162,2017-12-02,41.68,30110,51.343701,51.613123,18.45,18334,54.700600,48.942328,25.01,...,22.918940,46.031971,32.92,84600,65.071918,66.833321,8.0,473.149,104.9110865,NaN
2163,2017-12-03,41.70,30149,51.410204,51.309597,18.39,18182,54.247098,48.366501,25.01,...,22.804610,46.031971,32.87,84370,64.895008,66.617952,8.0,473.149,104.9110865,NaN
2164,2017-12-04,41.73,30208,51.510811,50.975377,18.38,18157,54.172509,48.151684,25.02,...,22.539714,46.031971,32.85,84280,64.825783,66.333359,8.0,473.149,104.9110865,NaN
2165,2017-12-05,41.74,30228,51.544915,50.637746,18.38,18157,54.172509,47.936868,25.03,...,22.352079,45.392013,32.80,84060,64.656565,66.479502,8.0,473.149,104.9110865,NaN
2166,2017-12-06,41.75,30248,51.579019,50.274538,18.34,18056,53.871170,47.289435,24.94,...,22.165693,45.264980,32.75,83830,64.479655,65.771864,8.0,473.149,104.9110865,NaN
2167,2017-12-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2168,2017-12-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2169,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2170,2017-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2171,2017-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looking at the last 20 rows of the dataset, we can see that the last actual date is '2017-12-06', even though there are a few more rows after that with dates but no data.

In [11]:
data.tail(20)

,date,Wemmershoek_height_(m),Wemmershoek_storage_(ml),Wemmershoek_current_%,Wemmershoek_last year_%,Steenbras Lower_height_(m),Steenbras Lower_storage_(ml),Steenbras Lower_current_%,Steenbras Lower_last year_%,Steenbras Upper_height_(m),...,Theewaterskloof_current_%,Theewaterskloof_last year_%,Berg River_height_(m),Berg River_storage_(ml),Berg River_current_%,Berg River_last year_%,Land-En Zeezicht_height_(m),Land-En Zeezicht_storage_(ml),Land-En Zeezicht_current_%,Land-En Zeezicht_last year_%
2147,2017-11-17,41.07,28921,49.316213,56.503649,18.47,18385,54.852761,54.247098,25.09,...,24.692620,48.059510,33.36,86620,66.625644,69.925390,8.0,473.149,104.9110865,NaN
2148,2017-11-18,41.07,28921,49.316213,56.188186,18.48,18410,54.927350,53.945759,25.07,...,24.490200,47.926645,33.25,86110,66.233367,69.779248,8.0,473.149,104.9110865,NaN
2149,2017-11-19,41.08,28941,49.350317,55.906828,18.46,18359,54.775189,53.569830,25.06,...,24.369414,47.926645,33.22,85930,66.094916,69.779248,8.0,473.149,104.9110865,NaN
2150,2017-11-20,41.09,28960,49.382716,55.557261,18.41,18233,54.399260,53.271474,25.05,...,24.289237,47.926645,33.15,85650,65.879548,69.779248,8.0,473.149,104.9110865,NaN
2151,2017-11-21,41.19,29152,49.710115,55.069572,18.45,18334,54.700600,52.904496,25.05,...,24.249044,47.463910,33.14,85610,65.848781,69.025460,8.0,473.149,104.9110865,NaN
2152,2017-11-22,41.36,29483,50.274538,54.721711,18.53,18536,55.303279,52.391324,25.09,...,24.249044,47.727765,33.25,86110,66.233367,68.917776,8.0,473.149,104.9110865,NaN
2153,2017-11-23,41.45,29657,50.571243,54.445468,18.56,18612,55.530030,51.949757,25.07,...,24.289237,47.529718,33.25,86110,66.233367,68.771633,8.0,473.149,104.9110865,NaN
2154,2017-11-24,41.50,29756,50.740059,54.203329,18.56,18612,55.530030,51.657368,25.05,...,24.209060,47.397894,33.28,86250,66.341051,68.448581,8.0,473.149,104.9110865,NaN
2155,2017-11-25,41.52,29795,50.806562,53.857172,18.54,18562,55.380852,51.436584,25.05,...,24.129299,47.266487,33.25,86110,66.233367,68.263980,8.0,473.149,104.9110865,NaN
2156,2017-11-26,41.55,29854,50.907169,53.683241,18.52,18511,55.228690,50.932363,25.04,...,23.930627,47.266487,33.21,85930,66.094916,68.048612,8.0,473.149,104.9110865,NaN


Since the last few rows aren't at all useful, I'll get rid of them

In [12]:
data = data[:2167]

In [215]:
bad_date_idx = data.loc[data['date'] > '2017-12-12'].index
max_bad_idx = max(bad_date_idx)
min_bad_idx = min(bad_date_idx)

print('The dates seem fine here...')
data.iloc[min_bad_idx-20:min_bad_idx]['date'].tolist()  
print('...The problem seems to start here...')
data.iloc[min_bad_idx:max_bad_idx+1]['date'].tolist()
print('...The problem seems to end there. Let\'s see the next few rows...')
data.loc[max_bad_idx+1:max_bad_idx+10]['date'].tolist()

The dates seem fine here...


[Timestamp('2017-04-29 00:00:00'),
 Timestamp('2017-04-30 00:00:00'),
 Timestamp('2017-01-05 00:00:00'),
 Timestamp('2017-02-05 00:00:00'),
 Timestamp('2017-03-05 00:00:00'),
 Timestamp('2017-04-05 00:00:00'),
 Timestamp('2017-05-05 00:00:00'),
 Timestamp('2017-06-05 00:00:00'),
 Timestamp('2017-07-05 00:00:00'),
 Timestamp('2017-08-08 00:00:00'),
 Timestamp('2017-09-05 00:00:00'),
 Timestamp('2017-10-05 00:00:00'),
 Timestamp('2017-11-05 00:00:00'),
 Timestamp('2017-12-05 00:00:00'),
 Timestamp('2017-05-13 00:00:00'),
 Timestamp('2017-05-14 00:00:00'),
 Timestamp('2017-05-15 00:00:00'),
 Timestamp('2017-05-16 00:00:00'),
 Timestamp('2017-05-17 00:00:00'),
 Timestamp('2017-05-18 00:00:00')]

...The problem seems to start here...


[Timestamp('2018-05-19 00:00:00'),
 Timestamp('2019-05-20 00:00:00'),
 Timestamp('2020-05-21 00:00:00'),
 Timestamp('2021-05-22 00:00:00')]

...The problem seems to end there. Let's see the next few rows...


[Timestamp('2017-05-23 00:00:00'),
 Timestamp('2017-05-24 00:00:00'),
 Timestamp('2017-05-25 00:00:00'),
 Timestamp('2017-05-26 00:00:00'),
 Timestamp('2017-05-27 00:00:00'),
 Timestamp('2017-05-28 00:00:00'),
 Timestamp('2017-05-29 00:00:00'),
 Timestamp('2017-05-30 00:00:00'),
 Timestamp('2017-05-31 00:00:00'),
 Timestamp('2017-06-01 00:00:00')]

In [218]:
(datetime.datetime(year = 2017, month = 12, day = 6) - data['date'].min()).days + 1
data.shape[0]

2167

2167

In [219]:
for idx, d in data['date'].iteritems():
    if idx > 0:
        data.iloc[idx, 0] = data.iloc[idx - 1, 0] + datetime.timedelta(days = 1)

In [220]:
data['date'].min()
data['date'].max()

Timestamp('2012-01-01 00:00:00')

Timestamp('2017-12-06 00:00:00')

# It's getting hot in here: melting our DataFrame

The data is 'wide' formatted (also known as 'pivoted'), with columns names repeated horizontally for dam. This isn't very easy for computers to read, so we'll need to do some wrangling to transform this into 'long' format. When we're done, there will be no duplicate column names but there will be an additional column to identify each dam. 

In [221]:
dam_names = set(header_0)
dam_names = [d.title() for d in dam_names if not d.startswith('Total Stored')]

cols = data.columns.tolist()[1:]
dates = data['date']

dam_levels = pd.DataFrame()
for dam in dam_names:
    next_dam = pd.DataFrame({'date': dates,
                            'dam_name': dam})
    for col in cols:
        if re.search(dam, col):
            measure = re.sub(dam+'_', '', col)
            next_dam[measure] = data[col].replace(' ', '', regex = True).replace('#VALUE!', np.nan, regex = True)
            next_dam[measure] = next_dam[measure].astype(float)    
    dam_levels = pd.concat([dam_levels, next_dam], axis = 0)

dam_names = [dam.lower() for dam in dam_names]

In [225]:
dam_levels = dam_levels.rename(columns = {'height_(m)': 'height_m', 
                            'storage_(ml)': 'storage_ml',
                            'last year (%)': 'lastyear_%'})

In [226]:
dam_levels.head()

,dam_name,date,height_m,storage_ml,current_%,last year_%
0,Lewis Gay,2012-01-01,13.9,128.3,76.4,NaN
1,Lewis Gay,2012-01-02,13.9,128.3,76.4,NaN
2,Lewis Gay,2012-01-03,13.9,128.3,76.4,NaN
3,Lewis Gay,2012-01-04,13.9,128.3,76.4,NaN
4,Lewis Gay,2012-01-05,13.9,128.3,76.4,NaN


# Writing to disk

In [236]:
dam_levels.to_csv('/Users/jonathan/Google Drive/Data Science/Projects/Dam Levels/data/Dam-levels-clean-20120101-20171206.csv',
                 index = False)